In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import NMF
from sklearn import decomposition
from scipy.stats import norm
from opnmf import model, logging
import opnmf
import random

In [29]:
def shifting_values_min(data) : 

    for col in data.columns:
    
        # Get column min 
        min_val = data[col].min()
        
        # Shift column by its min value
        data[col] = data[col] + abs(min_val)
        #After checking all of the columns were 0 in the min value
    
    return data

In [18]:
def normalization(data):
    
    # Create StandardScaler and fit 
    scaler = StandardScaler()
    scaler.fit(data)

    # Transform data
    normalized_data = scaler.transform(data)  
    normalized_data = pd.DataFrame(normalized_data, columns=data.columns)
    
    #Assay the normalizide was conducted or not
    if normalized_data[normalized_data < 0].sum().sum() == 0  : 
        data_array = normalized_data.to_numpy()
        
        return data_array
    
    else : 
        normalized_data = shifting_values_min(normalized_data)
        data_array = normalized_data.to_numpy()
        
        return data_array

In [39]:
def stability_analysis(data_array, n_components_range, num_perturbations = 10, init = 'nndsvd', tolerance = 0.00001, random_state=42):
    """
    Performs stability analysis for OPNMF by training models on perturbed data and evaluating component similarity.

    Args:
      data_array: The original data matrix.
      n_components_range: A range of values for the number of components (n_components) to evaluate.
      num_perturbations: The number of perturbed data matrices to generate.
      init: The initialization method for OPNMF (default: 'nndsvd').
      tolerance: The tolerance parameter for OPNMF (default: 0.0001).

    Returns:
      optimal_n_components: The optimal number of components based on stability analysis.
      stability_coefficients: A dictionary mapping each number of components to its stability coefficient.
      reconstruction_errors: A dictionary mapping each number of components to its reconstruction error.
    """
    if random_state:
        random.seed(random_state)
        np.random.seed(random_state)

    stability_coefficients = {}
    reconstruction_errors = {}

    # Train OPNMF on original data to get original components
    estimator = opnmf.model.OPNMF(n_components=max(n_components_range), init=init, tol=tolerance)
    estimator.fit(data_array)
    original_components = estimator.components_

    for n_components in n_components_range:
        similarities = []

        for _ in range(num_perturbations):
            # Generate perturbed data
            perturbed_data = data_array + norm(0, 0.01).rvs(size=data_array.shape)
            scaler = StandardScaler()
            scaler.fit(perturbed_data)

            # Transform data
            normalized_data = scaler.transform(perturbed_data) 
            normalized_data = pd.DataFrame(normalized_data, columns=pd.DataFrame(data_array).columns)
            
            for col in normalized_data.columns:
                # Get column min 
                min_val = normalized_data[col].min()

                # Shift column by its min value
                normalized_data[col] = normalized_data[col] + abs(min_val)

            perturbed_data_array = normalized_data.to_numpy()

            # Train OPNMF model on perturbed data
            estimator = opnmf.model.OPNMF(n_components=n_components, init=init, tol=tolerance)
            W = estimator.fit_transform(perturbed_data_array)
            H = estimator.components_

            # Compute similarity to original components
            similarity = compute_similarity(H, original_components)
            similarities.append(similarity)

        # Average similarities for this n_components
        average_similarity = np.mean(similarities)
        stability_coefficients[n_components] = average_similarity

        # Reconstruction error for this n_components
        reconstruction_error = np.linalg.norm(perturbed_data_array - np.dot(W, H), ord='fro')
        reconstruction_errors[n_components] = reconstruction_error

    # Find optimal n_components based on highest stability coefficient
    optimal_n_components = max(stability_coefficients, key=stability_coefficients.get)

    return optimal_n_components, stability_coefficients, reconstruction_errors

def compute_similarity(components1, components2, similarity_measure='cosine'):
    """
    Calculates the similarity between two sets of OPNMF components.

    Args:
      components1: A matrix of components (H) from the first OPNMF model.
      components2: A matrix of components (H) from the second OPNMF model.
      similarity_measure: The similarity measure to use ('cosine' or 'frobenius').

    Returns:
      similarity: The similarity score between the two sets of components.
    """

    if similarity_measure == 'cosine':
        similarity = np.mean(np.dot(components1, components2.T))
    elif similarity_measure == 'frobenius':
        similarity = np.linalg.norm(components1 - components2, ord='fro')
    else:
        raise ValueError(f"Invalid similarity measure: {similarity_measure}")

    return similarity


In [40]:
def stability_plot(stabilities, errors):
    # Extract data
    n_components = list(stabilities.keys()) 
    stability_values = list(stabilities.values())
    error_values = list(errors.values())

    # Plot 
    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('n_components')
    ax1.set_ylabel('Stability', color=color) 
    ax1.plot(n_components, stability_values, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  

    color = 'tab:blue'
    ax2.set_ylabel('Reconstruction Error', color=color)  
    ax2.plot(n_components, error_values, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.invert_yaxis()   

    fig.tight_layout()  
    plt.show()
    fig.savefig('stability_plot.png')

In [41]:
# CSV Reading
merge_file = pd.read_csv("merge.csv")
df = merge_file.copy()
column_get = df.columns[1:-1]
df_2 = df[column_get]
df = df_2


In [42]:
data_array = normalization(df)

In [43]:
data_array 

array([[3.15521502, 3.51111203, 4.04718404, ..., 1.97413043, 2.90267839,
        1.65037602],
       [4.02217214, 4.21959358, 4.61608765, ..., 1.47967261, 1.99999639,
        0.97011588],
       [3.14271957, 1.57344163, 3.17869491, ..., 2.94909204, 2.66087498,
        2.97751256],
       ...,
       [4.11509286, 4.41202888, 2.48192696, ..., 2.86597459, 2.0363084 ,
        2.31752112],
       [3.69933512, 0.99147465, 3.48878136, ..., 1.60695758, 2.81276293,
        2.33209339],
       [4.10077989, 2.96576768, 1.48680891, ..., 3.0903917 , 2.11232154,
        3.31717897]])

In [44]:
data_array = data_array # your data array
n_components_range = (2,4,6,8,10,12,14,16)  # or any other range you want to test
num_perturbations = 10  # or any other number of perturbations you want to perform

optimal_n, stabilities, errors = stability_analysis(data_array, n_components_range, num_perturbations)

print("Optimal number of components:", optimal_n)
print("Stability coefficients:", stabilities)
print("Reconstruction errors:", errors)

OPNMF did not converge with tolerance = 1e-05 under 50000 iterations
/Users/ahmadrezasohrabi/opt/anaconda3/lib/python3.9/site-packages/opnmf/logging.py:168: RuntimeWarning: OPNMF did not converge with tolerance = 1e-05 under 50000 iterations
  warnings.warn(msg, category=category)
OPNMF did not converge with tolerance = 1e-05 under 50000 iterations
/Users/ahmadrezasohrabi/opt/anaconda3/lib/python3.9/site-packages/opnmf/logging.py:168: RuntimeWarning: OPNMF did not converge with tolerance = 1e-05 under 50000 iterations
  warnings.warn(msg, category=category)
OPNMF did not converge with tolerance = 1e-05 under 50000 iterations
/Users/ahmadrezasohrabi/opt/anaconda3/lib/python3.9/site-packages/opnmf/logging.py:168: RuntimeWarning: OPNMF did not converge with tolerance = 1e-05 under 50000 iterations
  warnings.warn(msg, category=category)


KeyboardInterrupt: 

In [45]:
def stability_analysis_iterative(data_array, n_components_range, num_iterations=10000, num_perturbations=10, init='nndsvd', tolerance=0.00001, random_state=42):
    """
    Performs stability analysis for OPNMF by training models on perturbed data and evaluating component similarity iteratively.

    Args:
      data_array: The original data matrix.
      n_components_range: A range of values for the number of components (n_components) to evaluate.
      num_iterations: The number of iterations for stability analysis (default: 10).
      num_perturbations: The number of perturbed data matrices to generate.
      init: The initialization method for OPNMF (default: 'nndsvd').
      tolerance: The tolerance parameter for OPNMF (default: 0.0001).
      random_state: Seed for random number generation (default: 42).

    Returns:
      optimal_n_components: The optimal number of components based on stability analysis.
      stability_coefficients: A dictionary mapping each number of components to its stability coefficient.
      reconstruction_errors: A dictionary mapping each number of components to its reconstruction error.
    """
    if random_state:
        random.seed(random_state)
        np.random.seed(random_state)

    # Initialize dictionaries for results
    stability_coefficients = {}
    reconstruction_errors = {}

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}/{num_iterations}")

        # Train OPNMF on original data to get original components
        estimator = opnmf.model.OPNMF(n_components=max(n_components_range), init=init, tol=tolerance)
        estimator.fit(data_array)
        original_components = estimator.components_

        for n_components in n_components_range:
            similarities = []

            for _ in range(num_perturbations):
                # Generate perturbed data
                perturbed_data = data_array + norm(0, 0.01).rvs(size=data_array.shape)
                scaler = StandardScaler()
                scaler.fit(perturbed_data)

                # Transform data
                normalized_data = scaler.transform(perturbed_data)
                normalized_data = pd.DataFrame(normalized_data, columns=pd.DataFrame(data_array).columns)

                for col in normalized_data.columns:
                    # Get column min
                    min_val = normalized_data[col].min()

                    # Shift column by its min value
                    normalized_data[col] = normalized_data[col] + abs(min_val)

                perturbed_data_array = normalized_data.to_numpy()

                # Train OPNMF model on perturbed data
                estimator = opnmf.model.OPNMF(n_components=n_components, init=init, tol=tolerance)
                W = estimator.fit_transform(perturbed_data_array)
                H = estimator.components_

                # Compute similarity to original components
                similarity = compute_similarity(H, original_components)
                similarities.append(similarity)

            # Average similarities for this n_components
            average_similarity = np.mean(similarities)
            stability_coefficients.setdefault(n_components, []).append(average_similarity)

            # Reconstruction error for this n_components
            reconstruction_error = np.linalg.norm(perturbed_data_array - np.dot(W, H), ord='fro')
            reconstruction_errors.setdefault(n_components, []).append(reconstruction_error)

    # Calculate average stability coefficients and reconstruction errors over iterations
    for n_components in n_components_range:
        stability_coefficients[n_components] = np.mean(stability_coefficients[n_components])
        reconstruction_errors[n_components] = np.mean(reconstruction_errors[n_components])

    # Find optimal n_components based on highest average stability coefficient
    optimal_n_components = max(stability_coefficients, key=stability_coefficients.get)

    return optimal_n_components, stability_coefficients, reconstruction_errors


In [47]:
data_array = data_array # your data array
n_components_range = (2,4,6)  # or any other range you want to test
num_perturbations = 10  # or any other number of perturbations you want to perform

optimal_n, stabilities, errors = stability_analysis(data_array, n_components_range, num_perturbations)

print("Optimal number of components:", optimal_n)
print("Stability coefficients:", stabilities)
print("Reconstruction errors:", errors)

Optimal number of components: 2
Stability coefficients: {2: 134588.86707716552, 4: 95372.3467193578, 6: 77799.48405151603}
Reconstruction errors: {2: 220.9000833798355, 4: 207.8285656309462, 6: 202.7683930533579}


In [ ]:
Optimal number of components: 2
Stability coefficients: {2: 134588.86707716552, 4: 95372.3467193578, 6: 77799.48405151603}
Reconstruction errors: {2: 220.9000833798355, 4: 207.8285656309462, 6: 202.7683930533579}